In [1]:
import numpy as np
import pandas as pd
import scapy
import pprint
from collections import defaultdict
from IPy import IP as IPy
from manuf import manuf

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
df = pd.read_csv("init_data")
df.head()

,Unnamed: 0,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name,Device Type,Manufacturer Device Type
0,0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b\'\\x17\\x03\\x01\\x02 0<...,NaN,NaN,00180A,Cisco Meraki,Phone,Meraki Phone
1,1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\\x93\\x938\\x1f\\x0f\\xe...","{'52.201.158.18', '172.217.6.39', '104.86.220....",NaN,28F076,"Apple, Inc.",Computer,Apple Computer
2,2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,"Apple, Inc.",TV,Apple TV
3,3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]\'3\\x13\\x8c\\x93\\xb...",{'52.46.136.77'},NaN,44650D,Amazon Technologies Inc.,Echo,Amazon Technologies Echo
4,4,5c:aa:fd:4c:87:a0,NaN,NaN,{'54.208.201.219'},NaN,5CAAFD,"Sonos, Inc.",Speaker,Sonos Speaker


In [3]:
df.iloc[:,1:8]

,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b\'\\x17\\x03\\x01\\x02 0<...,NaN,NaN,00180A,Cisco Meraki
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\\x93\\x938\\x1f\\x0f\\xe...","{'52.201.158.18', '172.217.6.39', '104.86.220....",NaN,28F076,"Apple, Inc."
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,"Apple, Inc."
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]\'3\\x13\\x8c\\x93\\xb...",{'52.46.136.77'},NaN,44650D,Amazon Technologies Inc.
4,5c:aa:fd:4c:87:a0,NaN,NaN,{'54.208.201.219'},NaN,5CAAFD,"Sonos, Inc."
5,5c:aa:fd:4c:92:86,NaN,NaN,{'54.163.161.36'},NaN,5CAAFD,"Sonos, Inc."
6,78:28:ca:03:80:0c,NaN,"{'151.101.40.246': ""b'GET /audio/7fd0fe06a28db...",{'107.22.76.192'},NaN,7828CA,"Sonos, Inc."
7,84:38:35:5a:70:40,"googleads.g.doubleclick.net.,cm.g.doubleclick....","{'185.217.0.110': '', '104.154.127.116': 'b\'\...","{'172.217.6.38', '199.59.150.42', '172.217.6.3...",NaN,843835,"Apple, Inc."
8,88:71:e5:d2:73:4b,NaN,"{'104.154.126.239': ""b'\\xb2\\xf4\\xdf\\xc1\\x...","{'52.46.132.50', '34.236.127.105'}",NaN,8871E5,Amazon Technologies Inc.
9,b4:7c:9c:31:b4:df,"spectrum.s3.amazonaws.com.,ntp-g7g.amazon.com....","{'93.184.216.34': '', '52.216.17.168': '', '52...","{'54.239.27.116', '34.200.197.98', '54.239.27....","linux-6.local.,linux-6.local.,Android.local.,l...",B47C9C,Amazon Technologies Inc.


In [5]:
y1 = df['Manufacturer Device Type']
y2 = df['Device Type']
train_data = df.iloc[:,1:8]

In [4]:
#del df['Unnamed: 0']
#del df['Device Type']
#del df['Manufacturer Device Type']


In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

orgEncoder = LabelEncoder()
orgEncoder.fit(train_data['Organization Name'])

train_data["Organization Name"] = orgEncoder.transform(train_data['Organization Name'])
#test_data["DayOfWeek_Num"] = dayofweek_encoder.transform(test_data['DayOfWeek'])


In [7]:
train_data.head()

,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b\'\\x17\\x03\\x01\\x02 0<...,NaN,NaN,00180A,3
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\\x93\\x938\\x1f\\x0f\\xe...","{'52.201.158.18', '172.217.6.39', '104.86.220....",NaN,28F076,1
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,1
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]\'3\\x13\\x8c\\x93\\xb...",{'52.46.136.77'},NaN,44650D,0
4,5c:aa:fd:4c:87:a0,NaN,NaN,{'54.208.201.219'},NaN,5CAAFD,4


In [8]:
clf = RandomForestClassifier()
clf.fit(train_data, y1)
print(clf.feature_importances_)

ValueError: could not convert string to float: 'EC1A59'